In [64]:
import re
import graphviz

In [65]:
# COMMMENTS

# (1) : we assumed this pattern is valid : a*?  (https://regex101.com)
# (2) : ?? is assumed ? [ valid ]
# (3) : ++ is assumed + [ valid ]
# (4) : ** is invalid

#TODO:
# valid : ? || + || *
# else is not valid

open_list = ["[","("]
close_list = ["]",")"]
# Function to check parentheses
def check_braces(myStr):
    stack = []
    for i in myStr:
        if i in open_list:
            stack.append(i)
        elif i in close_list:
            pos = close_list.index(i)
            if ((len(stack) > 0) and
                (open_list[pos] == stack[len(stack)-1])):
                stack.pop()
            else:
                return False
    if len(stack) == 0:
        return True
    else:
        return False

def check_ORs(regex):
  ORs = regex.count('|')
  splitted = list(filter(lambda x: x, regex.split('|')))
  if(len(splitted) != ORs + 1):
    return False
  return True

def check_valid_question_mark(regex):
  matches = re.findall(r'\?{3,}', regex)
  if(len(matches) > 0):
    return False

  while(regex.find("??") != -1):
    regex = regex.replace("??", "?")

  if(regex[0] == '?'):
    return False

  if(regex.find('|?') != -1):
    return False

  return regex

def check_valid_plus(regex):
  matches = re.findall(r'\+{3,}', regex)
  if(len(matches) > 0):
    return False

  while(regex.find("++") != -1):
    regex = regex.replace("++", "+")

  if(regex[0] == '+'):
    return False

  if(regex.find('|+') != -1):
    return False

  return regex



def check_valid_star(regex):
  matches = re.findall(r'\*{2,}', regex)
  if(len(matches) > 0):
    return False

  if(regex[0] == '*'):
    return False

  if(regex.find('|*') != -1):
    return False

  return True


def validate_regex(regex):
  # check ORs (|)
  if (not check_ORs(regex)):
    return False

  # check braces
  if(not check_braces(regex)):
    return False

  # check ?
  regex = check_valid_question_mark(regex)
  if(not regex):
    return False

  # check +
  regex = check_valid_plus(regex)
  if(not regex):
    return False

  # check *
  if(not check_valid_star(regex)):
    return False

  pettern = "[a-zA-Z0-9\.\+\?\*\[\]\(\)\-\|]"
  matched = re.findall(pettern, regex)
  if(len(matched) != len(regex)):
    return False

  return True

In [66]:
def validateRegex(regex):
    try:
        re.compile(regex)
    except re.error:
        print(f"Invalid input: {regex}")
        return False
    return True

In [67]:
class postfixByShuntYard:
    def __init__(self, regex):
        self.regex = regex
        self.postfix = self.applyShuntYard(regex)

    def getPostfixedRegex(self):
        return self.postfix

    def applyShuntYard(self, regex):
        # *, +, ? : have the same precendence level
        # . : mid precendence level
        # | : lowest precedence level
        symbolPrecedence = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
        postfix = ""
        stack = []
        # If we find [xyz] as example, this would be converted to (x|y|z)
        for i in range(len(regex)):
            if regex[i] == '[':
                j = i + 1
                while regex[j] != ']':
                    if regex[j].isalnum() and regex[j + 1].isalnum():
                        regex = regex[:j + 1] + '|' + regex[j + 1:]
                    j += 1

        # Replace the [] with ()
        regex = regex.replace('[', '(')
        regex = regex.replace(']', ')')

        print("postfix1: ", regex)

        # If we find a-f would be converted to (a|b|c|d|e|f)
        for i in range(regex.count('-')):
            for j in range(len(regex)):
                character = regex[j]
                if character == '-':
                    end = regex[j + 1]
                    start = regex[j - 1]
                    range_list = ''
                    for k in range(int(ord(end) - ord(start))):
                        range_list = range_list + '|'
                        range_list = range_list + chr(ord(start) + k + 1)
                    regex = regex[0: j] + range_list + regex[j + 2:]
                    break
        print("postfix2: ", regex)

        # We use . to divide the regex to subRegex
        # example ab(b|c)*d+ will be converted to ab.bc|*.d+.
        dotLocations = []
        for i in range(len(regex) - 1):
            startSymbols = [')', "*","+", "*"]
            endSymbols = ["*", "+", ".", "|", ")"]
            if regex[i] in startSymbols and regex[i+1] not in endSymbols:
                dotLocations.append(i)
            elif regex[i].isalnum() and (regex[i+1].isalnum() or regex[i+1] == '('):
                dotLocations.append(i)

        for i in range(len(dotLocations)):
            regex = regex[:dotLocations[i] + 1 + i] + '.' + regex[dotLocations[i] + 1 + i:]
        print("postfix3: ", regex)

        # Applying Shunting Yard Algorithm
        for i in range(len(regex)):
            character = regex[i]
            # If ( -> push in stack
            if character == '(':
                stack.append(character)
            # If ) -> pop from stack until ( is found
            elif character == ')':
                while stack[-1] != '(':
                    postfix = postfix + stack[-1]
                    stack.pop()
                stack.pop()  # to remove ( from the stack

            # order the operators based on their precedence
            elif character in symbolPrecedence:
                while stack and symbolPrecedence.get(character, 0) <= symbolPrecedence.get(stack[-1], 0):
                    postfix = postfix + stack[-1]
                    stack.pop()
                stack.append(character)

            # If a normal character (not an symbol or parenthesis)
            else:
                postfix = postfix + character
        # pop any remaining symbols from the stack and appends them to the postfix string
        while stack:
            postfix = postfix + stack[-1]
            stack.pop()
        print("postfix5: ", regex)
        return postfix

In [68]:
class State:
    def __init__(self, name, transitions=[], parents=[], start=False, accepted=True):
        self.name = name
        self.parents = []
        self.transitions = []
        self.start = start
        self.accepted = accepted

    def transition(self, symbol, state):
        self.transitions.append((symbol, state))
        self.accepted = False
        state.parents.append(self)

    def parents(self):
        return self.parents.copy()

In [79]:
class NFA:
    def __init__(self, postfix=None, start=None, accept=None):
        self.start = start
        self.accept = accept
        if postfix and not start and not accept:
            obj = self.convertToNFA(postfix)
            self.start = obj.start
            self.accept = obj.accept

    def getStateByName(self, name):
        for state in self.getStates():
            if state.name == name:
                return state
        return None

    def getStatesByNames(self, names):
        names = names.split()
        result = []
        for name in names:
            for state in self.getStates():
                if state.name == name:
                    result.append(state)
        return result

    def isAcceptedState(self, states):
        for state in states:
            if state.accepted:
                return True
        return False

    def getStates(self):
        visited = set()
        states = []
        queue = [self.start]
        visited.add(self.start)
        while queue:
            state = queue.pop(0)
            states.append(state)
            for (transition) in state.transitions:
                if transition[1] not in visited:
                    visited.add(transition[1])
                    queue.append(transition[1])

        return states

    def getInputSymbols(self):
        states = self.getStates()
        symbols = set()
        for state in states:
            for symbol, next_state in state.transitions:
                if symbol != 'ϵ':
                    symbols.add(symbol)
        return list(symbols)

    def convertToNFA(self, postfix):
        stack = []
        stateCounter = 0
        for character in postfix:
            if character == '.':
                # state_1 is the old state in the stack
                # state_2 is the new state in the stack
                state_2 = stack.pop()
                state_1 = stack.pop()
                state_1.accept.transition('ϵ', state_2.start)
                stack.append(NFA(start = state_1.start, accept = state_2.accept))

            elif character == '*':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', accept)
                state_1.accept.transition('ϵ', start)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '|':
                state_2 = stack.pop()
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', state_2.start)
                state_1.accept.transition('ϵ', accept)
                state_2.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '+':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                state_1.accept.transition('ϵ', start)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '?':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', accept)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            else:
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition(character, accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

        return stack.pop()

    def JSONIFY(self):
        states = {}
        for state in self.getStates():
            stateObj = {
                'isAcceptedState': state.accepted,
            }
            for character, nextState in state.transitions:
                if character not in stateObj:   # if one transition
                    stateObj[character] = nextState.name
                else:   # if multiple transitions to the same state
                    stateObj[character] += ',' + nextState.name
            states[state.name] = stateObj

        return {
            'startingState': self.start.name,
            **states,
        }

    def visualize(self, name, view=False):
        json = self.JSONIFY()
        graph = graphviz.Digraph(engine='dot')
        for state, transitions in json.items():
            if state == 'startingState':
                continue
            if transitions['isAcceptedState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')
            for char, nextState in transitions.items():
                if char == 'isAcceptedState':
                    continue
                children = nextState.split(',')
                for child in children:
                    graph.edge(state, child, label=char)
        graph.render(name, view=view)
        return graph


In [80]:
class DFA:
    def __init__(self, nfa):
        self.nfa = nfa
        self.states = self.convertNFAtoDFA(nfa)

    def getStateByName(self, name):
        return self.nfa.getStateByName(name)

    def getStatesByName(self, names):
        return self.nfa.getStatesByName(names)

    def getInputSymbols(self):
        return self.nfa.getInputSymbols()

    def epsClosure(self, states):

        closure_set = set(states)
        stack_states = list(states)

        while stack_states:
            state = stack_states.pop()
            for input_symbol, next_state in state.transitions:
                # To avoid infinite loop
                if next_state not in closure_set:
                    if input_symbol == "ϵ":
                        closure_set.add(next_state)
                        stack_states.append(next_state)

        closure_list = list(closure_set)
        closure_list.sort(key=lambda x: x.name)
        closure_str = ''

        for state in closure_list:
            closure_str += ' ' + state.name

        # remove the first space
        return closure_str.strip()


    def transitionStep(self, nfa, states, input_symbol):
        move_states = set()
        states_list = states.split()
        states = []

        for name in states_list:
            states.append(nfa.getStateByName(name))

        for state in states:
            for in_s, next_state in state.transitions:
                if in_s == input_symbol:
                    move_states.add(next_state)

        return move_states

    def convertNFAtoDFA(self, nfa):
        nfa_states = nfa.getStates()
        input_symbols = nfa.getInputSymbols()

        start_state = self.epsClosure([nfa_states[0]])
        self.states = {'startingState': start_state}

        queue = [start_state]
        visited_states = set([start_state])

        while queue:
            current_state = queue.pop(0)
            for in_symbol in input_symbols:
                next_states = self.epsClosure(self.transitionStep(nfa, current_state, in_symbol))

                if next_states == '' or next_states == ' ':
                    continue

                if next_states not in visited_states:
                    queue.append(next_states)
                    visited_states.add(next_states)

                if current_state not in self.states:
                    self.states[current_state] = {}
                self.states[current_state][in_symbol] = next_states

            self.states.setdefault(current_state, {})['isAcceptedState'] = nfa.isAcceptedState(nfa.getStatesByNames(current_state))

        return self.states

    def toDict(self):
        return self.states.copy()

    def visualize(self, name='output/dfa.gv', view=False):

        graph = graphviz.Digraph(engine='dot')

        for state, transitions in self.states.items():
            if state == 'startingState':
                continue
            if transitions['isAcceptedState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')

            for char, next_state in transitions.items():

                if char == 'isAcceptedState':
                    continue
                children_states = next_state.split(',')

                for child in children_states:
                    graph.edge(state, child, label=char)
        graph.render("output/dfa", view=False)


In [81]:

cases = {0: 'ab(b|c)*d+', 1: ' (AB)', 2: '(A|B)', 3: '([A-Z])', 4: '(A)+', 5: '(A)*', 6: '(((AB)((A|B)*))(AB))', 7: '((((AB)|[A-Z])+)([A-Z]*))', 8: '(((((ABE)|C)|((([A-Z])S)*))+)((AB)C))',
          9: '((([a-z_])(([a-z0-9_])*))(([!?])?))', 10: '(A(((B*)|(DA))*))((CG)|(D([DEF])))', 11: '(ab', 12: '(a([b-c))', 13: '((a|b)|)', 14: '(a{3,2})'}
regex = cases[0]
# 7

# regex = input("Enter regular expression: ")
if not validateRegex(regex):
    print("non valiid")

postfix = postfixByShuntYard(regex)
print("----------------------------------------------------------------")
print("regex:", regex)
print("----------------------------------------------------------------")
print("postfix: ", postfix.getPostfixedRegex())
print("----------------------------------------------------------------")
nfa = NFA(postfix=postfix.getPostfixedRegex())
print("NFA: ", nfa.JSONIFY())
nfa.visualize(name='graphs/nfa_graph.gv', view=False)
print("----------------------------------------------------------------")
dfa = DFA(nfa)
print("DFA: ", dfa.toDict())
dfa.visualize(name='output/dfa.gv', view=False)
print("----------------------------------------------------------------")
# minDfa = MIN_DFA(dfa)
# print("Minimized DFA: ", minDfa.toDict())
# minDfa.visualize(name='output/min_dfa.gv', view=False)



postfix1:  ab(b|c)*d+
postfix2:  ab(b|c)*d+
postfix3:  a.b.(b|c)*.d+
postfix5:  a.b.(b|c)*.d+
----------------------------------------------------------------
regex: ab(b|c)*d+
----------------------------------------------------------------
postfix:  ab.bc|*.d+.
----------------------------------------------------------------
NFA:  {'startingState': 'S0', 'S0': {'isAcceptedState': False, 'a': 'S1'}, 'S1': {'isAcceptedState': False, 'ϵ': 'S2'}, 'S2': {'isAcceptedState': False, 'b': 'S3'}, 'S3': {'isAcceptedState': False, 'ϵ': 'S10'}, 'S10': {'isAcceptedState': False, 'ϵ': 'S8,S11'}, 'S8': {'isAcceptedState': False, 'ϵ': 'S4,S6'}, 'S11': {'isAcceptedState': False, 'ϵ': 'S14'}, 'S4': {'isAcceptedState': False, 'b': 'S5'}, 'S6': {'isAcceptedState': False, 'c': 'S7'}, 'S14': {'isAcceptedState': False, 'ϵ': 'S12'}, 'S5': {'isAcceptedState': False, 'ϵ': 'S9'}, 'S7': {'isAcceptedState': False, 'ϵ': 'S9'}, 'S12': {'isAcceptedState': False, 'd': 'S13'}, 'S9': {'isAcceptedState': False, 'ϵ': 'S1

In [54]:
import re
def validateRegex(regex):
    try:
        re.compile(regex)
    except re.error:
        print(f"Invalid regular expression: {regex}")
        return False
    return True
class POSTFIX:
    def __init__(self, regex):
        self.regex = regex
        self.postfix = self.shunt_yard(regex)
    def get_postfix(self):
        return self.postfix
    def shunt_yard(self, regex):
        # Map each operator to its precedence level.
        # The operators supported by this code are * (kleene star), + (one or more), ? (zero or one), . (concatenation), and | (alternation).
        operators = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
        # postfix will eventually store the postfix notation of the input regular expression,.
        # stack is used as an intermediate stack in the Shunting Yard algorithm.
        postfix, stack = "", ""
        # Check if the regular expression contains any character classes (denoted by square brackets).
        # If a character class is found, the function converts it to an alternation between the characters inside the class.
        # For example, the character class [abc] would be converted to the regular expression (a|b|c).
        # This is done using a while loop that iterates over the characters of the regular expression, finds the opening and closing brackets of the character class, and replaces the contents of the class with an alternation.
        for i in range(len(regex)):
            c = regex[i]
            if c == '[':
                j = i + 1
                while regex[j] != ']':
                    if regex[j].isalnum() and regex[j + 1].isalnum():
                        regex = regex[:j + 1] + '|' + regex[j + 1:]
                    j += 1

        # Replace all remaining square brackets with parentheses.
        # This is done because parentheses are used to group sub-expressions in regular expressions
        regex = regex.replace('[', '(')
        regex = regex.replace(']', ')')

        print("postfix1: ", regex)

        # Replace any hyphen character (-) in the regular expression with an alternation between the characters on either side of the hyphen.
        # For example, the expression a-z would be converted to (a|b|c|...|y|z).
        # This is done using another for loop that iterates over the characters of the regular expression, finds the hyphen character, and replaces it with an alternation.
        hyphen_count = regex.count('-')
        for i in range(hyphen_count):
            for j in range(len(regex)):
                c = regex[j]
                if c == '-':
                    final = regex[j + 1]
                    first = regex[j - 1]
                    temp_list = ''
                    for k in range(int(ord(final) - ord(first))):
                        temp_list = temp_list + '|'
                        char = chr(ord(first) + k + 1)
                        temp_list = temp_list + char
                    regex = regex[0: j] + temp_list + regex[j + 2:]
                    break
        print("postfix2: ", regex)
        # Insert a concatenation operator (.) between any two adjacent characters(characters that are not operators), unless the characters are already separated by an operator, or the second character is an opening parenthesis.
        dotIndices = []
        for i in range(len(regex) - 1):
            startOps = [')', "*","+", "*"]
            endOps = ["*", "+", ".", "|", ")"]
            if regex[i] in startOps and regex[i+1] not in endOps:
                dotIndices.append(i)
            elif regex[i].isalnum() and (regex[i+1].isalnum() or regex[i+1] == '('):
                dotIndices.append(i)

        for i in range(len(dotIndices)):
            regex = regex[:dotIndices[i] + 1 + i] + '.' + regex[dotIndices[i] + 1 + i:]
        print("postfix3: ", regex)
        # Iterate over each character of the regular expression and performs the Shunting Yard algorithm.
        for i in range(len(regex)):
            c = regex[i]
            # If the character is an opening parenthesis, push it onto the stack.
            if c == '(':
                stack = stack + c
            # If the character is a closing parenthesis, pop operators off the stack and append them to the postfix string until an opening parenthesis is found. Then pop the opening parenthesis from the stack.
            elif c == ')':
                while stack[-1] != '(':
                    # places the character at the end of the stack in the postfix expression
                    postfix = postfix + stack[-1]
                    # [:-1] denotes up to or including the last character
                    stack = stack[:-1]
                stack = stack[:-1]  # removes the open bracket in the stack

            # If the character is an operator, pop operators off the stack and append them to the postfix string as long as they have higher or equal precedence to the current operator. Then push the current operator onto the stack.
            elif c in operators:
                while stack and operators.get(c, 0) <= operators.get(stack[-1], 0):
                    postfix, stack = postfix + stack[-1], stack[:-1]
                stack = stack + c

            # If the character is a operand (i.e. not an operator or parenthesis), append it to the postfix string.
            else:
                postfix = postfix + c
        # After iterating over all characters of the regular expression, the function pops any remaining operators off the stack and appends them to the postfix string.
        while stack:
            postfix, stack = postfix + stack[-1], stack[:-1]
        print("postfix5: ", regex)

        # Finally, the function returns the postfix notation of the input regular expression.
        return postfix

In [55]:
class State:
    def __init__(self, label, transitions=[], parents=[], is_start=False, is_accept=True):
        self.label = label
        self.parents = []
        self.transitions = []
        self.is_start = is_start
        self.is_accept = is_accept

    def add_transition(self, symbol, state):
        self.transitions.append((symbol, state))
        self.is_accept = False
        state.parents.append(self)

    def get_parents(self):
        return self.parents.copy()

In [57]:
import graphviz
class NFA:
    def __init__(self, start=None, accept=None, postfix=None):
        self.start = start
        self.accept = accept
        if not start and not accept and postfix:
            obj = self.postfix2nfa(postfix)
            self.start = obj.start
            self.accept = obj.accept
    # utility functions
    def getStateByLabel(self, label):
        for state in self.getStates():
            if state.label == label:
                return state
        return None
    def getStatesByLabel(self, labels):
        labels = labels.split()
        statesList = []
        for label in labels:
            for state in self.getStates():
                if state.label == label:
                    statesList.append(state)
        return statesList

    def checkIfAcceptingState(self, states):
        for state in states:
            if state.is_accept:
                return True
        return False

    def getStates(self):
        visited = set()
        states = []
        queue = [self.start]
        visited.add(self.start)
        while queue:
            state = queue.pop(0)
            states.append(state)
            for (transition) in state.transitions:
                if transition[1] not in visited:
                    visited.add(transition[1])
                    queue.append(transition[1])

        return states
    def getSymbols(self):
        """
        Returns the set of symbols that can be used to transition between NFA states.
        """
        states = self.getStates()
        symbols = set()
        for state in states:
            for symbol, next_state in state.transitions:
                if symbol != 'ϵ':
                    symbols.add(symbol)
        # convert to list of symbols
        return list(symbols)
    # Conversion from postfix to NFA
    def postfix2nfa(self, postfix): # shunting yard algorithm
        nfaStack = []
        i = 0
        for c in postfix:
            if c == '*':
                nfa1 = nfaStack.pop()
                start, accept = State('S' + str(i)), State('S' + str(i + 1))
                start.add_transition('ϵ', nfa1.start)
                start.add_transition('ϵ', accept)
                nfa1.accept.add_transition('ϵ', start)
                nfa1.accept.add_transition('ϵ', accept)
                nfaStack.append(NFA(start, accept))
                i += 2

            elif c == '.':
                nfa2, nfa1 = nfaStack.pop(), nfaStack.pop()
                nfa1.accept.add_transition('ϵ', nfa2.start)
                nfaStack.append(NFA(nfa1.start, nfa2.accept))

            elif c == '|':
                nfa2, nfa1 = nfaStack.pop(), nfaStack.pop()
                start, accept = State('S' + str(i)), State('S' + str(i + 1))
                start.add_transition('ϵ', nfa1.start)
                start.add_transition('ϵ', nfa2.start)
                nfa1.accept.add_transition('ϵ', accept)
                nfa2.accept.add_transition('ϵ', accept)
                nfaStack.append(NFA(start, accept))
                i += 2

            elif c == '+':
                nfa1 = nfaStack.pop()
                start, accept = State('S' + str(i)), State('S' + str(i + 1))
                start.add_transition('ϵ', nfa1.start)
                nfa1.accept.add_transition('ϵ', start)
                nfa1.accept.add_transition('ϵ', accept)
                nfaStack.append(NFA(start, accept))
                i += 2

            elif c == '?':
                nfa1 = nfaStack.pop()
                start, accept = State('S' + str(i)), State('S' + str(i + 1))
                start.add_transition('ϵ', nfa1.start)
                start.add_transition('ϵ', accept)
                nfa1.accept.add_transition('ϵ', accept)
                nfaStack.append(NFA(start, accept))
                i += 2

            else:
                start, accept = State('S' + str(i)), State('S' + str(i + 1))
                start.add_transition(c, accept)
                nfaStack.append(NFA(start, accept))
                i += 2

        return nfaStack.pop()
    # For exporting NFA
    def toDict(self):
        states = {}
        for state in self.getStates():
            state_dict = {
                'isTerminatingState': state.is_accept,
            }
            for char, next_state in state.transitions:
                if char not in state_dict:
                    state_dict[char] = next_state.label
                else:
                    state_dict[char] += ',' + next_state.label
            states[state.label] = state_dict

        return {
            'startingState': self.start.label,
            **states,
        }
    def visualize(self, name='output/nfa.gv', view=False):
        nfa_json = self.toDict()
        graph = graphviz.Digraph(engine='dot')
        for state, transitions in nfa_json.items():
            if state == 'startingState':
                continue
            if transitions['isTerminatingState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')
            for char, next_state in transitions.items():
                if char == 'isTerminatingState':
                    continue
                children_states = next_state.split(',')
                for child in children_states:
                    graph.edge(state, child, label=char)
        graph.render(name, view=view)
        return graph


In [58]:
import graphviz
from collections import deque


class DFA:
    def __init__(self, nfa):
        self.nfa = nfa
        self.states = self.nfa2dfa(nfa)
    # utility functions
    def getStateByLabel(self, label):
        return self.nfa.getStateByLabel(label)
    def getStatesByLabel(self, labels):
        return self.nfa.getStatesByLabel(labels)
    def getSymbols(self):
        return self.nfa.getSymbols()
    # DFA functions
    def epsilonClosure(self, states):
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            for symbol, next_state in state.transitions:
                if next_state not in closure:
                    if symbol == "ϵ":
                        closure.add(next_state)
                        stack.append(next_state)
        # convert set to list
        closureList =  list(closure)
        # sort the list
        closureList.sort(key=lambda x: x.label)
        # convert into a string representation
        closureString = ''
        for state in closureList:
            closureString += ' ' + state.label
        # remove the first space
        return closureString[1:]
    def move(self, nfa, states, symbol):
        move_states = set()
        # convert string to list
        statesList = states.split()
        # convert list of labels to list of states
        states = []
        for label in statesList:
            states.append(nfa.getStateByLabel(label))
        for state in states:
            for s, next_state in state.transitions:
                if s == symbol:
                    move_states.add(next_state)
        return move_states
    # Conversion from NFA to DFA
    def nfa2dfa(self, nfa):
        nfaStates = nfa.getStates()
        symbols = nfa.getSymbols()
        # initial state
        start_state = self.epsilonClosure([nfaStates[0]])
        self.states = {'startingState': start_state}
        # the rest of the nfaStates
        queue = deque([start_state])
        seen = set([start_state])
        while queue:
            current_state = queue.popleft()
            for symbol in symbols:
                next_states = self.epsilonClosure(self.move(nfa, current_state, symbol))
                if next_states == '' or next_states == ' ':
                    continue
                if next_states not in seen:
                    queue.append(next_states)
                    seen.add(next_states)
                self.states.setdefault(current_state, {})[symbol] = next_states
            self.states.setdefault(current_state, {})['isTerminatingState'] = nfa.checkIfAcceptingState(nfa.getStatesByLabel(current_state))
        return self.states
    # for exporting the DFA
    def toDict(self):
        return self.states.copy()
    def visualize(self, name='output/dfa.gv', view=False):
        graph = graphviz.Digraph(engine='dot')
        for state, transitions in self.states.items():
            if state == 'startingState':
                continue
            if transitions['isTerminatingState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')
            for char, next_state in transitions.items():
                if char == 'isTerminatingState':
                    continue
                children_states = next_state.split(',')
                for child in children_states:
                    graph.edge(state, child, label=char)
        graph.render("output/dfa", view=False)




In [59]:
import graphviz


class MIN_DFA:
    def __init__(self, dfa):
        self.dfa = dfa
        self.states = self.dfa2min(dfa)
    # Utility functions
    def getGroupKeys(self, group):
        keys = []
        for state in group:
            for key, value in state.items():
                keys.append(key)
        return keys
    # COnversion from DFA to Minimized DFA
    def dfa2min(self, dfa):
        states = dfa.toDict()
        symbols = dfa.getSymbols()
        # remove the final states from dfa
        states.pop('startingState')
        groups = []
        final_states = []
        non_final_states = []
        # iterate over states keys and values
        for key, value in states.items():
            # the value is a dictionary with the transitions and the isTerminatingState flag
            if value["isTerminatingState"] == True:
                final_states.append({key: value})
            else:
                non_final_states.append({key: value})
        groups.append(final_states)
        groups.append(non_final_states)
        #---------------------------------------------------------------------------
        # for each group in groups, check if there is a state that has a transition to a state in another group
        # if there is, split the group into two groups
        #---------------------------------------------------------------------------
        split = True
        while split:
            split = False # [[{key1: {a: key1, b: key2}}, {key2: {a: w}}] [{key3: {a: key1, b: key2}}, {}]]
            for i, group in enumerate(groups): # [{key1: {a: key1, b: key2}}, {key2: {a: key3}}] [{key3: {a: key1, b: key2}}, {}]
                if not group:
                    continue
                targetGroups = {}
                first_state = next(iter(group)) # {key1: {a: key1, b: key2}}
                for key, value in first_state.items():
                    for symbol in symbols:
                        if symbol in value:
                            targetGroups[symbol] = [j for j, group in enumerate(groups) if value[symbol] in self.getGroupKeys(group)][0] # {key1, key2}
                splitted_states = []
                for k, state in enumerate(group):
                    outputGroups = {}
                    for key, value in state.items():
                        for symbol in symbols:
                            if symbol in value:
                                List = [j for j, group in enumerate(groups) if value[symbol] in self.getGroupKeys(group)]
                                outputGroups[symbol] = List[0]
                    if outputGroups != targetGroups:
                        split = True
                        splitted_states.append(state)
                if len(splitted_states) > 0:
                    groups.insert(i+1, list(splitted_states))
                    groups[i] = [state for state in group if state not in splitted_states]
        newGroups = self.concatStates(groups)
        return newGroups
    def concatStates(self, groups):
        # create a hashtable for the states with the group number as the key
        hashTable = {}
        for g, group in enumerate(groups):
            # for all the states in the group, rename them to the group number
            for state in group:
                for key, value in state.items():
                    hashTable[key] = str(g)
        # Create a new dictionary for the new groups
        newGroups = {'startingState':0}
        groupCopy = groups.copy()
        # iterate over the groups
        for g, group in enumerate(groupCopy):
            # iterate over the states in the group
            for state in group:
                # iterate over the states in the group
                for key, value in state.items():
                    # iterate over the transitions in the state
                    for symbol, next_state in value.items():
                        # if the next state is in another group, replace it with the group number
                        if next_state in hashTable:
                            value[symbol] = str(hashTable[next_state])
                            newGroups[str(g)] = value
        return newGroups
    # For exporting the minimized DFA
    def toDict(self):
        return self.states
    def visualize(self, name='output/min.gv', view=False):
        graph = graphviz.Digraph(engine='dot')
        for state, transitions in self.states.items():
            if state == 'startingState':
                continue
            if transitions['isTerminatingState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')
            for char, next_state in transitions.items():
                if char == 'isTerminatingState':
                    continue
                children_states = next_state.split(',')
                for child in children_states:
                    graph.edge(state, child, label=char)
        graph.render("output/min", view=False)






In [61]:


def main():
    cases = {0: 'ab(b|c)*d+', 1: ' (AB)', 2: '(A|B)', 3: '([A-Z])', 4: '(A)+', 5: '(A)*', 6: '(((AB)((A|B)*))(AB))', 7: '((((AB)|[A-Z])+)([A-Z]*))', 8: '(((((ABE)|C)|((([A-Z])S)*))+)((AB)C))',
             9: '((([a-z_])(([a-z0-9_])*))(([!?])?))', 10: '(A(((B*)|(DA))*))((CG)|(D([DEF])))', 11: '(ab', 12: '(a([b-c))', 13: '((a|b)|)', 14: '(a{3,2})'}
    regex = cases[0]
    # 7

    # regex = input("Enter regular expression: ")
    if not validateRegex(regex):
        return
    try:
        postfix = POSTFIX(regex)
        print("----------------------------------------------------------------")
        print("regex:", regex)
        print("----------------------------------------------------------------")
        print("postfix: ", postfix.get_postfix())
        print("----------------------------------------------------------------")
        nfa = NFA(postfix=postfix.get_postfix())
        print("NFA: ", nfa.toDict())
        nfa.visualize(name='output/nfa.gv', view=False)
        print("----------------------------------------------------------------")
        dfa = DFA(nfa)
        print("DFA: ", dfa.toDict())
        dfa.visualize(name='output/dfa.gv', view=False)
        print("----------------------------------------------------------------")
        minDfa = MIN_DFA(dfa)
        print("Minimized DFA: ", minDfa.toDict())
        minDfa.visualize(name='output/min_dfa.gv', view=False)
    # catch the excpetion anr print it
    except Exception as e:
        print(e)
        print("Your Regex may be invalid")


if __name__ == '__main__':
    main()


postfix1:  ab(b|c)*d+
postfix2:  ab(b|c)*d+
postfix3:  a.b.(b|c)*.d+
postfix5:  a.b.(b|c)*.d+
----------------------------------------------------------------
regex: ab(b|c)*d+
----------------------------------------------------------------
postfix:  ab.bc|*.d+.
----------------------------------------------------------------
NFA:  {'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S1'}, 'S1': {'isTerminatingState': False, 'ϵ': 'S2'}, 'S2': {'isTerminatingState': False, 'b': 'S3'}, 'S3': {'isTerminatingState': False, 'ϵ': 'S10'}, 'S10': {'isTerminatingState': False, 'ϵ': 'S8,S11'}, 'S8': {'isTerminatingState': False, 'ϵ': 'S4,S6'}, 'S11': {'isTerminatingState': False, 'ϵ': 'S14'}, 'S4': {'isTerminatingState': False, 'b': 'S5'}, 'S6': {'isTerminatingState': False, 'c': 'S7'}, 'S14': {'isTerminatingState': False, 'ϵ': 'S12'}, 'S5': {'isTerminatingState': False, 'ϵ': 'S9'}, 'S7': {'isTerminatingState': False, 'ϵ': 'S9'}, 'S12': {'isTerminatingState': False, 'd': 'S13'}, 'S